In [1]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
# Load unit path data
unit_path = pd.read_csv('../unit_path.csv')

In [3]:
def generate_graph():
    G = nx.DiGraph()

    for index, row in unit_path.iterrows():
        source = row['domain']  
        target = row['range']   
        edge = row['P']         
        weight = row['W']       

        G.add_node(source)        
        G.add_node(target)

        if source in G and target in G[source].keys():
            if weight < G[source][target]['weight']:
                G.remove_edge(source, target)
            else: continue
            
        G.add_edge(source, target, property=edge, weight=weight, label=edge)  # 엣지 추가
    
    return G

In [4]:
G = generate_graph()

In [5]:
source_node = 'schema.MusicAlbum'
target_node = 'schema.Country'

In [18]:
def has_property(path, p_value):
    if p_value == None:
        return True
    
    p_values = []
    for i in range(len(path) - 1):
        source_node = path[i]
        target_node = path[i + 1]
        edge_data = G[source_node][target_node]
        p_values.append(edge_data['property'])
    return (p_value in p_values)